WE USED THIS FILE FOR TEST AND GETTING THE HISTORICAL DATA

In [1]:
import requests
import credentials
import pandas as ps

In [2]:
session = requests.Session()

In [3]:
instrument = "EUR_USD"
count = 10
granularity = "H1"

In [4]:
# this pattern is taken from the documentation of oanda site and it is better to look for it yourself
url = f"{credentials.OANDA_URL}/instruments/{instrument}/candles"

In [5]:
url

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

In [6]:
param = dict(
    count=count,
    granularity=granularity,
    price="MBA"
)

In [7]:
param

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

In [8]:
API_KEY = "388d8d5440ad979dd6e1745b1f566bde-a5dea5109879fe5f64d0c8b871ab59e8"
header = f"Bearer {API_KEY}"
response = session.get(url, params=param, headers=credentials.SECURE_HEADER)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [11]:
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [12]:
len(data['candles'])

10

In [13]:
for candle in data['candles']:
    print(candle)

{'complete': True, 'volume': 5252, 'time': '2022-06-20T14:00:00.000000000Z', 'bid': {'o': '1.05179', 'h': '1.05410', 'l': '1.05174', 'c': '1.05343'}, 'mid': {'o': '1.05186', 'h': '1.05418', 'l': '1.05182', 'c': '1.05350'}, 'ask': {'o': '1.05192', 'h': '1.05426', 'l': '1.05191', 'c': '1.05358'}}
{'complete': True, 'volume': 3193, 'time': '2022-06-20T15:00:00.000000000Z', 'bid': {'o': '1.05342', 'h': '1.05376', 'l': '1.05230', 'c': '1.05373'}, 'mid': {'o': '1.05350', 'h': '1.05383', 'l': '1.05237', 'c': '1.05380'}, 'ask': {'o': '1.05358', 'h': '1.05390', 'l': '1.05244', 'c': '1.05388'}}
{'complete': True, 'volume': 2244, 'time': '2022-06-20T16:00:00.000000000Z', 'bid': {'o': '1.05373', 'h': '1.05375', 'l': '1.05186', 'c': '1.05227'}, 'mid': {'o': '1.05381', 'h': '1.05383', 'l': '1.05193', 'c': '1.05234'}, 'ask': {'o': '1.05389', 'h': '1.05391', 'l': '1.05200', 'c': '1.05242'}}
{'complete': True, 'volume': 963, 'time': '2022-06-20T17:00:00.000000000Z', 'bid': {'o': '1.05226', 'h': '1.0522

In [23]:
print(data['candles'][0])

{'complete': True, 'volume': 5252, 'time': '2022-06-20T14:00:00.000000000Z', 'bid': {'o': '1.05179', 'h': '1.05410', 'l': '1.05174', 'c': '1.05343'}, 'mid': {'o': '1.05186', 'h': '1.05418', 'l': '1.05182', 'c': '1.05350'}, 'ask': {'o': '1.05192', 'h': '1.05426', 'l': '1.05191', 'c': '1.05358'}}


In [14]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [15]:
# this is to understand the logic of what we're going to do later
data['candles'][0]['bid']['o']

'1.05179'

In [16]:
for price in prices:
    for oh in ohlc:
        print(f"{price}-{oh}")

mid-o
mid-h
mid-l
mid-c
bid-o
bid-h
bid-l
bid-c
ask-o
ask-h
ask-l
ask-c


In [17]:
our_data = []
for candle in data['candles']:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]
    our_data.append(new_dict)
print(our_data[0])

{'time': '2022-06-20T14:00:00.000000000Z', 'volume': 5252, 'mid_o': '1.05186', 'mid_h': '1.05418', 'mid_l': '1.05182', 'mid_c': '1.05350', 'bid_o': '1.05179', 'bid_h': '1.05410', 'bid_l': '1.05174', 'bid_c': '1.05343', 'ask_o': '1.05192', 'ask_h': '1.05426', 'ask_l': '1.05191', 'ask_c': '1.05358'}


In [18]:
import pandas as pd

In [19]:
candles_dataframe = pd.DataFrame.from_dict(our_data)
candles_dataframe

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-20T14:00:00.000000000Z,5252,1.05186,1.05418,1.05182,1.05350,1.05179,1.05410,1.05174,1.05343,1.05192,1.05426,1.05191,1.05358
1,2022-06-20T15:00:00.000000000Z,3193,1.05350,1.05383,1.05237,1.05380,1.05342,1.05376,1.05230,1.05373,1.05358,1.05390,1.05244,1.05388
2,2022-06-20T16:00:00.000000000Z,2244,1.05381,1.05383,1.05193,1.05234,1.05373,1.05375,1.05186,1.05227,1.05389,1.05391,1.05200,1.05242
3,2022-06-20T17:00:00.000000000Z,963,1.05233,1.05233,1.05060,1.05060,1.05226,1.05226,1.05050,1.05051,1.05240,1.05241,1.05068,1.05069
4,2022-06-20T18:00:00.000000000Z,1459,1.05062,1.05074,1.04990,1.04992,1.05053,1.05067,1.04981,1.04983,1.05070,1.05082,1.04997,1.05000
5,2022-06-20T19:00:00.000000000Z,809,1.04996,1.05122,1.04981,1.05114,1.04987,1.05112,1.04972,1.05105,1.05004,1.05131,1.04989,1.05123
6,2022-06-20T20:00:00.000000000Z,796,1.05119,1.05146,1.05090,1.05112,1.05109,1.05136,1.05080,1.05099,1.05129,1.05156,1.05099,1.05124
7,2022-06-20T21:00:00.000000000Z,286,1.05088,1.05150,1.05086,1.05116,1.05049,1.05100,1.05044,1.05078,1.05128,1.05200,1.05120,1.05155
8,2022-06-20T22:00:00.000000000Z,1095,1.05117,1.05188,1.05090,1.05184,1.05075,1.05180,1.05075,1.05175,1.05159,1.05196,1.05099,1.05194


In [20]:
candles_dataframe.to_pickle('EUR_USD_H1.pkl')

In [21]:
df = pd.read_pickle('EUR_USD_H1.pkl')

In [22]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-06-20T14:00:00.000000000Z,5252,1.05186,1.05418,1.05182,1.05350,1.05179,1.05410,1.05174,1.05343,1.05192,1.05426,1.05191,1.05358
1,2022-06-20T15:00:00.000000000Z,3193,1.05350,1.05383,1.05237,1.05380,1.05342,1.05376,1.05230,1.05373,1.05358,1.05390,1.05244,1.05388
2,2022-06-20T16:00:00.000000000Z,2244,1.05381,1.05383,1.05193,1.05234,1.05373,1.05375,1.05186,1.05227,1.05389,1.05391,1.05200,1.05242
3,2022-06-20T17:00:00.000000000Z,963,1.05233,1.05233,1.05060,1.05060,1.05226,1.05226,1.05050,1.05051,1.05240,1.05241,1.05068,1.05069
4,2022-06-20T18:00:00.000000000Z,1459,1.05062,1.05074,1.04990,1.04992,1.05053,1.05067,1.04981,1.04983,1.05070,1.05082,1.04997,1.05000
5,2022-06-20T19:00:00.000000000Z,809,1.04996,1.05122,1.04981,1.05114,1.04987,1.05112,1.04972,1.05105,1.05004,1.05131,1.04989,1.05123
6,2022-06-20T20:00:00.000000000Z,796,1.05119,1.05146,1.05090,1.05112,1.05109,1.05136,1.05080,1.05099,1.05129,1.05156,1.05099,1.05124
7,2022-06-20T21:00:00.000000000Z,286,1.05088,1.05150,1.05086,1.05116,1.05049,1.05100,1.05044,1.05078,1.05128,1.05200,1.05120,1.05155
8,2022-06-20T22:00:00.000000000Z,1095,1.05117,1.05188,1.05090,1.05184,1.05075,1.05180,1.05075,1.05175,1.05159,1.05196,1.05099,1.05194
